# <a id='toc1_'></a>[create dataset](#toc0_)

**Table of contents**<a id='toc0_'></a>    
- [create dataset](#toc1_)    
  - [init code](#toc1_1_)    
  - [load data](#toc1_2_)    
    - [option clean csv](#toc1_2_1_)    
    - [ (deprecated) option dirty csv](#toc1_2_2_)    
  - [arrange data](#toc1_3_)    
    - [trim column names](#toc1_3_1_)    
    - [set date column](#toc1_3_2_)    
    - [relabeling duplicate columns](#toc1_3_3_)    
    - [remove duplicates or invalids](#toc1_3_4_)    
    - [parse dates to datetime 🕥](#toc1_3_5_)    
    - [(deprecated) remove columns w/ no or few values](#toc1_3_6_)    
  - [condensed dataset](#toc1_4_)    
    - [define functions](#toc1_4_1_)    
    - [define single, multi and misc columns](#toc1_4_2_)    
    - [concat condensed dataset from subsets](#toc1_4_3_)    
    - [create manual table for mapping](#toc1_4_4_)    
    - [column mapping from codebook](#toc1_4_5_)    
    - [value mapping for missings and unknown](#toc1_4_6_)    
    - [value mapping from codebook](#toc1_4_7_)    
  - [💾 data for pivot use](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[init code](#toc0_)

In [1]:

import io
import os
from glob import glob
import pandas as pd
from pathlib import Path
from pandas_plots import tbl, pls, hlp
from pandas_plots.hlp import add_bitmask_label
import re
hlp.show_package_version(["pygwalker"])

os.environ["THEME"]="light"

dir_data_in=Path("../data/in")
dir_data_out=Path("../data/out")
dir_data_codebooks=Path("../data/codebooks")

# ! only 1 csv file in in/ is allowed
file_csv_list=glob(f"{dir_data_in}/*.csv")
assert len(file_csv_list) == 1, f"❌ Expected 1 csv file, found {len(file_csv_list)}"

file_csv = file_csv_list[0]

# * grab choice part or eg .1
REGEX_CHOICE = r"( \(choice=.*?\)+|\.\d$)"

# * grab the code block at the start
REGEX_CODE = r"(^\[.*?\])"

# * grab the non-code block
REGEX_NONCODE = r"\].*"

ENABLE_UPDATE=True

🐍 3.12.2 | 📦 pygwalker: 0.4.9.14 | 📦 pandas: 2.2.3 | 📦 numpy: 1.26.4 | 📦 duckdb: 1.2.1 | 📦 pandas-plots: 0.12.22 | 📦 connection_helper: 0.8.15


## <a id='toc1_2_'></a>[load data](#toc0_)


### <a id='toc1_2_1_'></a>[option clean csv](#toc0_)

In [2]:
df_csv = pd.read_csv(
    file_csv,
    sep=",",
    encoding="utf-8-sig",
    dtype={"[01.01] Bewertung - CTCAE-Bewertung": str},
)

### <a id='toc1_2_2_'></a>[ (deprecated) option dirty csv](#toc0_)
- ⚠️ strategy to import the csv raw data:
  - ❌ do not use default `pd.read_csv()`, its not as error tolerant as excel
  - ✅ read csv as `windows-1252` string and cleanse it
    - most (not all) lines have heading + trailing quotes, remove
    - data have a mixture of "" and ", downgrade by 1
    - delimiter "," also occurs in quotes, change to ";" only outside of quotes
    - now byte stream can be imported

In [3]:
# # * replace delimiter *only* outside of quotes
# def replace_delimiter_outside_quotes(
#     input: str, delimiter_old: str = ",", delimiter_new: str = ";", quotechar: str = '"'
# ):
#     outside_quotes = True
#     output = ""
#     # * loop through input and toggle inside/outside status
#     for char in input:
#         if char == quotechar:
#             outside_quotes = not outside_quotes
#         elif outside_quotes and char == delimiter_old:
#             char = delimiter_new
#         output += char
#     return output

# # * downgrades quotes in string to normalize quots: " -> NA, "" -> "
# def reduce_quotes_in_string(input: str):
#     output = ""
#     len_=len(input)
#     i = 0
#     while i < len_:
#         # * check current + next char if within range
#         if input[i] == '"' and i+1 < len_ and input[i+1] != '"':
#             output += ''
#             i = i+1
#         elif input[i] == '"'and i+1 < len_ and input[i+1] == '"':
#             output+= '"'
#             i = i+2
#         else:
#             output += input[i]
#             i=i+1
#     return output

# # * read csv -> variable
# with open(file_csv, encoding="windows-1252") as f:
#     csv=f.read()

# # * selectively change , into ;
# csv_lines = csv.splitlines()
# csv_lines_new = []
# for line in csv_lines[:]:
#     # * remove first and last quotes if present
#     if line[:1] == '"':
#         line = line[1:-1]
#     # * tailor line by line
#     line=reduce_quotes_in_string(line)
#     line=replace_delimiter_outside_quotes(line)
#     # * just now quotes can be removed, not before
#     line=line.replace('"', '')
#     csv_lines_new.append(line)
#     # print(f'{line.count(";")} {line}')

# # * list -> string fo feed into df constructor
# csv_string = "\n".join(csv_lines_new).replace('\t', '')

# df_csv = pd.read_csv(
#     io.StringIO(csv_string),
#     sep=";",
#     encoding="windows-1252",
# )

## <a id='toc1_3_'></a>[arrange data](#toc0_)

### <a id='toc1_3_1_'></a>[trim column names](#toc0_)

In [4]:
cols_trimmed = [col.strip() for col in df_csv.columns]
df_csv.columns = cols_trimmed

### <a id='toc1_3_2_'></a>[set date column](#toc0_)

In [5]:
[print(col) for col in df_csv.columns if "Datum" in col]
col_date = "[02.01] Basics - Datum"

[02.01] Basics - Datum


### <a id='toc1_3_3_'></a>[relabeling duplicate columns](#toc0_)

In [6]:
df_csv.rename(columns={"[03.05.01] Folgen - Wie oft wurde die lebensnotwendige Intervention wiederholt?":"[03.05.02] Folgen - Wie oft wurde die lebensnotwendige Intervention wiederholt?"}, inplace=True)

### <a id='toc1_3_4_'></a>[remove duplicates or invalids](#toc0_)

In [7]:
# # * remove known erroruos lines
df_csv.drop([0,34,33], inplace=True, axis=0, errors='ignore')
# * remove "löschen" lines
df_csv = df_csv[~(df_csv[col_date] == "löschen")]

# * remove "Anmerkung"
df_csv.drop(['[05.09] Rahmenbedingungen - Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=5 Anmerkung)'], axis=1, inplace=True, errors='ignore')

### <a id='toc1_3_5_'></a>[parse dates to datetime 🕥](#toc0_)

In [8]:
# %%script echo skipping

# * manual override of invalid dates
df_csv.loc[df_csv[col_date]=="2023_08_2028", col_date]="2023_08_28"
df_csv.loc[df_csv[col_date]=="2024_16", col_date]="2024_06"

# * split column into  all possible date patterns, these may not interfere!
# * parse date col
REG_DATE_YYYY_MM=r"\d{4}_\d{2}$"
REG_DATE_YYYY_MM_DD=r"\d{4}_\d{2}_\d{2}$"
REG_DATE_DD_MM_YYYY=r"\d{2}.\d{2}.\d{4}$"

# * get subsets according to regex
_df1=df_csv[df_csv[col_date].str.contains(REG_DATE_YYYY_MM, na=False)][col_date]
_df2=df_csv[df_csv[col_date].str.contains(REG_DATE_YYYY_MM_DD, na=False)][col_date]
_df3=df_csv[df_csv[col_date].str.contains(REG_DATE_DD_MM_YYYY, na=False)][col_date]

# * convert each to corresponding datetime
_df1=pd.to_datetime(_df1, format="%Y_%m").astype(str)
_df2=pd.to_datetime(_df2, format="%Y_%m_%d").astype(str)
_df3=pd.to_datetime(_df3, format="%d.%m.%Y").astype(str)

# * update in place
df_csv.update(_df1)
df_csv.update(_df2)
df_csv.update(_df3)

# * now convert the whole col
df_csv[col_date]=pd.to_datetime(df_csv[col_date], format='ISO8601')

# * blend timestamp into date column
df_csv[col_date].fillna(df_csv["Survey Timestamp"], inplace=True)

### <a id='toc1_3_6_'></a>[(deprecated) remove columns w/ no or few values](#toc0_)

In [9]:
# import numpy as np
# # * remove columns with no value
# cols_no_value = [col for col in df_csv.columns if df_csv[col].isna().all()]
# print("these columns have no values at all:")

# print(np.array(cols_no_value))

# # ! remove columns
# cols_after_removal = list(set(df_csv.columns) - set(cols_no_value))
# df_csv_short = df_csv[list(set(df_csv.columns) - set(cols_no_value))]

In [10]:
df_csv

,Participant ID,Survey Identifier,Survey Timestamp,[02.01] Basics - Datum,[01.01] Bewertung - CTCAE-Bewertung,[01.02] Wann ist das AE erstmals aufgetreten?,"[01.03] Bewertung - Glauben Sie, dass das AE mit dem Sport assoziiert ist?",[02.02] Basics - Was war das AE? (choice=Schmerzen),[02.02] Basics - Was war das AE? (choice=Übelkeit/ Erbrechen),[02.02] Basics - Was war das AE? (choice=Oberflächliche Verletzungen),...,Welche motorische Hauptform dominierte die Bewegungseinheit ? (choice=Schnelligkeit).1,In welcher Phase der geplanten Sporteinheit trat das AE auf?.1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Gut).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Durchschnittlich).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Mäßig).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Weiß nicht).1,Welchen Trainingszustand hatte der / die Betroffene zum Zeitpunkt als das AE auftrat? (choice=Anmerkung).1,"Glaubst Du, dass das AE mit dem Sport assoziiert ist?",Anmerkung.1,Complete?.2
1,3,NaN,2022-10-13 13:41:16,2022-10-01,2,Das AE war vor dem Sport bereits vorhanden und...,No,Unchecked,Checked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
2,4,NaN,2022-11-08 09:19:23,2022-10-01,1,Das AE ist neu aufgetreten.,Yes,Checked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
3,5,NaN,2022-11-08 09:31:05,2022-11-01,2,Das AE war vor dem Sport bereits vorhanden und...,No,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
4,6,NaN,2022-11-18 10:10:55,2022-11-01,1,Das AE ist neu aufgetreten.,No,Unchecked,Checked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
5,7,NaN,2022-11-21 14:02:25,2022-11-01,3,Das AE ist neu aufgetreten.,Yes,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,204,NaN,NaN,2024-11-27,1,Das AE ist neu aufgetreten.,Yes,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
194,205,NaN,NaN,2024-12-03,1,Das AE war vor dem Sport bereits vorhanden und...,Yes,Unchecked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
195,206,NaN,NaN,2024-12-08,2,Das AE ist neu aufgetreten.,Yes,Unchecked,Checked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN
196,207,NaN,NaN,2024-12-12,1,Das AE ist neu aufgetreten.,Yes,Checked,Unchecked,Unchecked,...,Unchecked,NaN,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,NaN,NaN,NaN


## <a id='toc1_4_'></a>[condensed dataset](#toc0_)
- take the cleansed (but structurally unchanged) dataset `df_csv` as a starting point
- define handling for column subsets according to column types
  - **single choice** - only 1 value is allowed per row
  - **multi choice** - multiple values are allowed per row
  - **misc** - all other columns that are selected to appear in the final dataset
- apply column mapping from manual table `codebook_columns.xlsx`
- apply value mapping for missings and unknown value
- apply value mapping from manual table `codebook_values.xlsx`

### <a id='toc1_4_1_'></a>[define functions](#toc0_)

In [11]:
def condense_columns_single_choice(df, strict=False):
    """
    Condenses multiple columns into a single target column, extracting specific values based on
    the first occurrence of 'Checked' in each row. 

    The function processes columns named according to a pattern (e.g., 'choice=<value>') and condenses
    them into a single column. If a column contains the value 'Checked', the corresponding value (e.g., 
    'Schmerzen', 'Muskelkater') is assigned to the new target column. If multiple 'Checked' values are 
    present, only the leftmost occurrence is used.

    Parameters:
        df (pandas.DataFrame): The input DataFrame with columns to be processed. The columns should contain
                                values such as 'Checked' or 'Unchecked', and be named according to a 
                                pattern like '[02.02] Basics - Was war das AE?  (choice=<value>)'.
        strict: if an error is raised if more than 1 cell is "Checked"

    Returns:
        pandas.DataFrame: A DataFrame with the same columns as input, but with a new target column containing 
                        the extracted values based on the first occurrence of 'Checked' in each row.
                        The new column is named using the pattern found in the column names (e.g., '[02.02]').

    Notes:
        - If no 'Checked' value is found in a row, the target column is set to pd.NA for that row.
        - The new column name is derived from the first part of the first input column (i.e., the substring
        within the square brackets).
        - Only the first 'Checked' occurrence in each row is considered; any subsequent 'Checked' cells
        will be ignored.
    """

    # * Extract target column name using the updated regex pattern (substring in square brackets)
    match = re.search(r"\[\d{2}\.\d{2}(\.\d{2})?\]", df.columns[0])
    if not match:
        raise ValueError("No substring in square brackets matching the pattern found in the first column name.")
    target_col_name = match.group(0)

    # * Check for multiple "checked" cells in one row
    columns_to_check = [col for col in df.columns if re.search(r"\(choice=.*\)", col)]
    checked_counts = df[columns_to_check].apply(lambda row: (row.str.lower() == "checked").sum(), axis=1)

    if strict and (checked_counts > 1).any():
        problematic_rows = df.index[checked_counts > 1].tolist()
        raise ValueError(f"Multiple 'checked' values found in rows: {problematic_rows}")

    # * Create the target column with default pd.NA values
    df[target_col_name] = pd.NA

    # * Process each row and select the first "checked" value from left to right
    for index, row in df.iterrows():
        # * Loop over columns left to right to find the first "checked" value
        for col in columns_to_check:
            if row[col].lower() == "checked":
                # * Extract the choice label using regex. capture group is 
                match = re.search(r"\(choice=(.*?)\)", col)
                # match = re.search(r"\(choice=\d+ (.*?)\)", col)
                if match:
                    choice_label = match.group(1)
                    # Set the target column with the choice label for this row
                    df.at[index, target_col_name] = choice_label
                break  # Stop after the first "checked" value is found

    return df.iloc[:,-1]

def condense_columns_multi_choice(
    df, 
    new_col_pattern=r"^\[.*?\]", 
    choice_pattern=r"\(choice=(.*?)\)"
):
    # Extract the new column name from the first column using the provided regex pattern
    first_col = df.columns[0]
    new_col_name = re.search(new_col_pattern, first_col).group(0) if re.search(new_col_pattern, first_col) else "Unnamed"
    
    # Extract parts from original column names using the provided choice pattern
    extracted_parts = [
        re.search(choice_pattern, col).group(1) if re.search(choice_pattern, col) else None
        for col in df.columns
    ]

    # Calculate the new column using a bitmask
    def calculate_bitmask(row):
        bitmask = 0
        for i, col in enumerate(df.columns):
            if row[col] == "Checked":
                bitmask += 2**i
        return bitmask

    df[new_col_name] = df.apply(calculate_bitmask, axis=1)
    
    # Example placeholder for a function to add additional labels or transformations
    # Replace `add_bitmask_label` with your actual logic if needed
    # add_bitmask_label(df, new_col_name, extracted_parts)

    add_bitmask_label(df, new_col_name, extracted_parts)
    return df.iloc[:,-1]


### <a id='toc1_4_2_'></a>[define single, multi and misc columns](#toc0_)

In [12]:

stubs_single = set((
'[03.01.02]',
'[03.14.02]', '[03.15.01]',
))


stubs_multi = set((
'[02.02]', '[02.03]', '[02.04]', 
'[03.03.02]', '[03.03.01]', '[03.06.01]',
'[03.07.01]', '[03.08.01]', '[03.11.01]', '[03.12.01]', '[03.14.01]',
'[03.17.01]', '[03.14]',
'[05.07]', '[05.09]',
))

stubs_misc= set((
'Participant ID', 
'[01.02]','[01.03]', '[03.01]', '[03.02]', '[03.03]',
'[03.04]', '[03.05]', '[03.06]', '[03.07]', '[03.08]', '[03.09]', '[03.10]',
'[03.11]', '[03.12]', '[03.13]', '[03.15]', '[03.16]', '[03.17]', '[05.01]',
'[05.02]', '[05.03]', '[05.04]', '[05.05]', '[05.06]', '[05.08]',
'[03.01.01]', '[03.02.01]',
'[03.02.02]',
'[03.06.02]', '[03.07.02]', '[03.08.02]',
'[03.09.01]', '[03.09.02]', '[03.10.01]', '[03.10.02]', '[03.11.02]',
'[03.12.02]', '[03.13.01]', '[03.13.02]', 
'[03.15.02]', '[03.17.02]',

'[02.01]', '[01.01]', '[01.02]', '[03.01]',
'[03.02]', '[03.03]', '[03.04]', '[03.07]',
'[03.08]', '[03.08.02]',
'[03.09]', '[03.09.01]', '[03.09.02]', '[03.10]', '[03.11]', '[03.12]',
'[03.13]', '[03.13.01]', '[03.13.02]', '[03.17]', '[05.01]', '[05.02]',
'[05.03]', '[05.04]', '[05.05]', '[05.06]', '[05.08]',
'[03.18]'
))

if stubs_misc & stubs_multi or stubs_misc & stubs_single or stubs_single & stubs_multi:
    raise ValueError("stubs must be disjoint")

cols_single = hlp.find_cols(df_csv.columns, stubs_single)
cols_multi = hlp.find_cols(df_csv.columns, stubs_multi)
cols_misc = hlp.find_cols(df_csv.columns, stubs_misc)


In [13]:
# # * testing
# df_csv.find_cols(stubs=["NAOK"])

In [14]:
# # * derive single
df_csv_single = pd.DataFrame()
for stub in stubs_single:
    cols = [col for col in df_csv.columns if stub in col]
    print(cols)

    df = pd.DataFrame(df_csv[cols])
    ser = condense_columns_single_choice(df)
    df_csv_single[stub] = ser

df_csv_single

['[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=Schmerzen am gleichen Tag)', '[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=Schmerzen bis zum Folgetag)', '[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=Schmerzen, die mindestens 3 Tage anhielten)', '[03.01.02] Folgen - Wie lange dauerten die Schmerzen an? (choice=weiß nicht)']
['[03.14.02] Folgen - Wie lange wurde die Anpassung vorgenommen? (choice=nur für diese Einheit)', '[03.14.02] Folgen - Wie lange wurde die Anpassung vorgenommen? (choice=für die gesamte Therapiephase)', '[03.14.02] Folgen - Wie lange wurde die Anpassung vorgenommen? (choice=ab jetzt für alle bewegungseinheiten mit allen Patient*innen)']
['[03.15.01] Folgen - War die gesamte Handlungsweise bei diesem AE rein pädagogischer Natur? (choice=ja)', '[03.15.01] Folgen - War die gesamte Handlungsweise bei diesem AE rein pädagogischer Natur? (choice=nein)']


,[03.01.02],[03.14.02],[03.15.01]
1,<NA>,<NA>,<NA>
2,Schmerzen am gleichen Tag,nur für diese Einheit,<NA>
3,<NA>,<NA>,<NA>
4,<NA>,<NA>,<NA>
5,"Schmerzen, die mindestens 3 Tage anhielten",<NA>,ja
...,...,...,...
193,<NA>,<NA>,<NA>
194,<NA>,<NA>,<NA>
195,<NA>,<NA>,<NA>
196,<NA>,<NA>,<NA>


In [15]:
# # * derive multi
df_csv_multi = pd.DataFrame()
for stub in stubs_multi:
    cols = [col for col in df_csv.columns if stub in col]
    # print(cols)

    df = pd.DataFrame(df_csv[cols])
    ser = condense_columns_multi_choice(df)
    df_csv_multi[stub] = ser

df_csv_multi[:3]

,[03.11.01],[03.08.01],[05.09],[03.17.01],[03.14],[03.07.01],[05.07],[02.04],[02.02],[03.06.01],[03.12.01],[02.03],[03.14.01],[03.03.01],[03.03.02]
1,-,bei dem Betroffenen selbst,2 Durchschnittlich,-,-,-,Koordination,Internistisch,Übelkeit/ Erbrechen,-,-,Physische Belastung|Med. Therapie,-,-,-
2,-,bei dem Betroffenen selbst|im Behandlungsteam,1 Gut,-,-,-,Koordination,Rücken|Gesäß,Schmerzen,-,-,Koordinationsprobleme,Übungsauswahl|Intensität,-,-
3,-,bei dem Betroffenen selbst,3 Mäßig,-,-,-,Koordination,Internistisch,Kreislaufprobleme,-,-,Med. Therapie,-,Medikamentengabe oral,Innerhalb der ersten 24 Stunden


### <a id='toc1_4_3_'></a>[concat condensed dataset from subsets](#toc0_)

In [16]:
import re

# * save original columns as txt
cols_all = sorted(cols_single + cols_multi + cols_misc)

if ENABLE_UPDATE:
    pd.DataFrame(cols_all).to_csv(dir_data_out/"df_csv_condensed_columns.txt", index=False, sep=";", header=False)

# * all columns
df_csv_condensed = pd.concat(
    [
        df_csv[cols_misc],
        df_csv_single,
        df_csv_multi,
    ], 
    axis=1,
)
display(df_csv_condensed[:3])

# # ! prevent duplicate column names
if df_csv_condensed.columns.duplicated().any():
    raise ValueError("duplicated columns")

,[03.06] Folgen - Kam es zu einem erhöhten Pflegebedarf?,[03.11.02] Folgen - Wie lange wurde die PECH-Regel angewendet?,[03.12.02] Folgen - Wie lange wurde obeserviert?,[03.06.02] Folgen - Über welchen Zeitraum hinweg bestand der erhöhte Pflegebedarf?,Participant ID,[03.16] Folgen - Kam es zum Tod?,[03.02.01] Folgen - Wie zeitnah anch dem AE wurde entschieden einen Krankehausaufenthalt anzustreben oder zu verlängern?,[03.13.02] Folgen - Wie lange wurde pausiert?,[03.13] Folgen - Wurde die Bewegungseinheit gestoppt?,[03.17.02] Folgen - Wie lange?,...,[03.07.01],[05.07],[02.04],[02.02],[03.06.01],[03.12.01],[02.03],[03.14.01],[03.03.01],[03.03.02]
1,No,NaN,NaN,NaN,3,No,NaN,NaN,Yes,NaN,...,-,Koordination,Internistisch,Übelkeit/ Erbrechen,-,-,Physische Belastung|Med. Therapie,-,-,-
2,No,NaN,NaN,NaN,4,No,NaN,max. 5min,Yes,NaN,...,-,Koordination,Rücken|Gesäß,Schmerzen,-,-,Koordinationsprobleme,Übungsauswahl|Intensität,-,-
3,No,NaN,NaN,NaN,5,No,NaN,NaN,No,NaN,...,-,Koordination,Internistisch,Kreislaufprobleme,-,-,Med. Therapie,-,Medikamentengabe oral,Innerhalb der ersten 24 Stunden


### <a id='toc1_4_4_'></a>[create manual table for mapping](#toc0_)

In [17]:
import re

if ENABLE_UPDATE and False:

    # * have all columns but remove choices from column names, isolate code
    df_csv_cols = (
        pd.DataFrame(df_csv.columns, columns=["col"])
        .assign(
            col_code=lambda x: x["col"].str.extract(REGEX_CODE),
            col_clean=lambda x: x["col"].str.replace(REGEX_CHOICE, "", regex=True),
        )
    )
    display(df_csv_cols[:3])

    # * group and set up foe manual renaming
    df_csv_cols_condensed = (df_csv_cols
        .assign(cnt=1)
        .groupby(["col_clean","col_code"], as_index=False, dropna=False)["cnt"].sum()
    )
    display(df_csv_cols_condensed[:3])


    df_csv_cols_condensed.assign(col_new=lambda x: x["col_code"]).to_excel(dir_data_out/"codebook_columns.xlsx", index=False)
    # df_csv_cols_condensed.assign(col_new=lambda x: x["col_code"]).to_excel(dir_data_out/"df_csv_cols.ods", index=False, engine="odf")


### <a id='toc1_4_5_'></a>[column mapping from codebook](#toc0_)
- in `codebook_columns.xlsx`
  - col names in `col_clean` (full label) -> `col_final`
  - col names in `col_code` (code only) -> `col_new`

In [18]:
print(df_csv_condensed.columns.sort_values())

Index(['Participant ID', '[01.01] Bewertung - CTCAE-Bewertung',
       '[01.02] Wann ist das AE erstmals aufgetreten?',
       '[01.03] Bewertung - Glauben Sie, dass das AE mit dem Sport assoziiert ist?',
       '[02.01] Basics - Datum', '[02.02]', '[02.03]', '[02.04]',
       '[03.01.01] Folgen - Schmerzskala', '[03.01.02]',
       '[03.01] Folgen - Sind Schmerzen aufgetreten?',
       '[03.02.01] Folgen - Wie zeitnah anch dem AE wurde entschieden einen Krankehausaufenthalt anzustreben oder zu verlängern?',
       '[03.02.02] Folgen - Wie lange dauerte der (verlängerte) Krankenhausaufenthalt an?',
       '[03.02] Folgen - Kam es zu einem Krankenhausaufenthalt oder einer Verlängerung eines solchen?',
       '[03.03.01]', '[03.03.02]',
       '[03.03] - Folgen - Kam es zu einer medizinischen Weiterbehandlung?',
       '[03.04] Folgen - Kam es zu einer deutlichen Verzögerung bei der Einhaltung des medizinischen Therapieprotokolls?',
       '[03.05] Folgen - Kam es zu lebensnotwendigen In

In [19]:
# # * import column mapping
df_codebook_columns = (pd
        .read_excel(dir_data_codebooks/"codebook_columns.xlsx")
        # * remove leading and trailing spaces
        .applymap(lambda x: x.strip() if isinstance(x, str) else x)
)
# * ignore empty rows
df_codebook_columns = df_codebook_columns.dropna(subset=["col_final"])

# * coalesce columns - most right column has priority
# * -> deprecated. only use col_final!
# df_csv_cols_import["col_final"] = df_csv_cols_import[["col_new", "col_code", "col_clean"]].backfill(axis=1).iloc[:,0]

# * create dict for mapping
# * column names in target df can be full label or code only, so 2 rounds are needed
def create_col_map(df, join_col: str):
    # * only use rows with not NA values
    series_source = df_csv_condensed.columns.to_series()
    series_target = df_codebook_columns.set_index(join_col)["col_final"]
    # * use concat to join two series
    series_joint = (pd.concat(
        [
            series_source,
            series_target
        ],
        axis=1, join="inner"
        )
        ["col_final"]
    )
    return series_joint.to_dict()

col_map1 = create_col_map(df_codebook_columns, "col_clean")
col_map2 = create_col_map(df_codebook_columns, "col_code")

# * relabel column names in 2 rounds
df_csv_condensed.rename(columns=col_map1, inplace=True)
df_csv_condensed.rename(columns=col_map2, inplace=True)


# * (deprecated) strip column names to code only
# cols = df_csv_condensed.columns
# df_csv_condensed.columns = [re.sub(REGEX_NONCODE, "]", col) for col in cols]

df_csv_condensed

,[03.06] Increased care needs,[03.11.02] Folgen - Wie lange wurde die PECH-Regel angewendet?,[03.12.02] Folgen - Wie lange wurde obeserviert?,[03.06.02] Folgen - Über welchen Zeitraum hinweg bestand der erhöhte Pflegebedarf?,Participant ID,[03.16] Death,[03.02.01] Folgen - Wie zeitnah anch dem AE wurde entschieden einen Krankehausaufenthalt anzustreben oder zu verlängern?,[03.13.02] Folgen - Wie lange wurde pausiert?,[03.13] Stop,[03.17.02] Folgen - Wie lange?,...,[03.07.01],[05.07] Main motor skill,[02.04] Affected body parts,[02.02] Type,[03.06.01],[03.12.01],[02.03] Trigger,[03.14.01],[03.03.01],[03.03.02]
1,No,NaN,NaN,NaN,3,No,NaN,NaN,Yes,NaN,...,-,Koordination,Internistisch,Übelkeit/ Erbrechen,-,-,Physische Belastung|Med. Therapie,-,-,-
2,No,NaN,NaN,NaN,4,No,NaN,max. 5min,Yes,NaN,...,-,Koordination,Rücken|Gesäß,Schmerzen,-,-,Koordinationsprobleme,Übungsauswahl|Intensität,-,-
3,No,NaN,NaN,NaN,5,No,NaN,NaN,No,NaN,...,-,Koordination,Internistisch,Kreislaufprobleme,-,-,Med. Therapie,-,Medikamentengabe oral,Innerhalb der ersten 24 Stunden
4,No,NaN,NaN,NaN,6,No,NaN,min. 10min,Yes,NaN,...,-,Koordination,Internistisch,Übelkeit/ Erbrechen,-,-,Physische Belastung|Med. Therapie,-,-,-
5,Yes,über 7 Tage,NaN,über 1 Woche,7,No,NaN,NaN,Yes,NaN,...,Analgetika (Schmerzmittel,Koordination|Kraft,untere Extremitäten,Knochenverletzungen,Psychosoziale Unterstützung (Emotionale Unters...,-,"Stolpern, Ausrutschen, Fallen",-,Manuelle Untersuchung|Invasive Untersuchung|Me...,Innerhalb der ersten 24 Stunden
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,No,NaN,NaN,NaN,204,No,NaN,NaN,Yes,NaN,...,-,Ganzkörpertraining,Ganzkörper,Kreislaufprobleme,-,-,Physische Belastung,-,-,-
194,No,NaN,NaN,NaN,205,No,NaN,NaN,Yes,NaN,...,-,Koordination,Ganzkörper,Kreislaufprobleme,-,-,Physische Belastung,-,-,-
195,No,NaN,NaN,NaN,206,Yes,NaN,NaN,Yes,mindestens 3 Tage,...,-,Beweglichkeit,Ganzkörper,Übelkeit/ Erbrechen,-,-,Psychische Belastung|Med. Therapie,-,-,-
196,No,NaN,NaN,NaN,207,No,NaN,NaN,Yes,NaN,...,-,Ganzkörpertraining,Darm,Schmerzen|Stuhlgang,-,-,Physische Belastung|Med. Therapie,-,-,-


### <a id='toc1_4_6_'></a>[value mapping for missings and unknown](#toc0_)

In [20]:
# # * target coding for missing / unknown
STR_NA = pd.NA
STR_U = "U"

items = [
    ("[02.04]","-",STR_NA), 
    ("[03.02]","weiß nicht",STR_U),
    ("[03.03]","weiß nicht",STR_U),
    ("[03.12]","Weiß nicht",STR_U),
    ("[03.14.01]","-",STR_NA),
    ("[03.17]","Weiß nicht",STR_U),
    ("[05.03]","Weiß nicht",STR_U),
    ("[05.07]","-",STR_NA),
    ("[05.09]","-",STR_NA),
    ("[05.09]","Weiß nicht",STR_U),
    ("[03.08.01]","-",STR_NA),
]
for item in items:
    print(item)
    _col = df_csv_condensed.find_cols([item[0]])[0]
    # _col = df_csv_condensed.find_cols(["[02.04]"])[0]
    df_csv_condensed.loc[df_csv_condensed[_col]==item[1],_col] = item[2]
    # print(df_csv_condensed[_col].value_counts(dropna=False))

# * remove digits in [05.09]. inplace wont work, whyever
df_csv_condensed[df_csv_condensed.find_cols(["[05.09]"])] = df_csv_condensed[df_csv_condensed.find_cols(["[05.09]"])].replace(r"\d{1} ","", regex=True, inplace=False)

('[02.04]', '-', <NA>)
('[03.02]', 'weiß nicht', 'U')
('[03.03]', 'weiß nicht', 'U')
('[03.12]', 'Weiß nicht', 'U')
('[03.14.01]', '-', <NA>)
('[03.17]', 'Weiß nicht', 'U')
('[05.03]', 'Weiß nicht', 'U')
('[05.07]', '-', <NA>)
('[05.09]', '-', <NA>)
('[05.09]', 'Weiß nicht', 'U')
('[03.08.01]', '-', <NA>)


### <a id='toc1_4_7_'></a>[value mapping from codebook](#toc0_)
- in `codebook_values.xlsx`
  - col codes in `code`
  - old value in `de` -> new value in `en`

In [21]:
def replace_vectorized(df_source, df_target):
    # * Build the column-wise mapping: {col: {val_old: val_new}}
    mapping = df_source.groupby('col').apply(
        lambda g: dict(zip(g['val_old'], g['val_new']))
    ).to_dict()

    return df_target.replace(mapping)

# todo add to pandas-plots
import re
from typing import Any
def replace_substring_vectorized(df_mapping: pd.DataFrame, df_target: pd.DataFrame) -> pd.DataFrame:
    """
    Replaces substrings in specified columns of `df_target` based on a mapping defined in `df_mapping`.

    Each row in `df_mapping` must contain:
    - 'col': the name of the column in `df_target`
    - 'val_old': the substring to search for (case-insensitive, trimmed)
    - 'val_new': the substring to replace with (trimmed)

    Parameters:
        df_mapping (pd.DataFrame): A DataFrame containing columns 'col', 'val_old', and 'val_new'.
        df_target (pd.DataFrame): The target DataFrame where replacements will be applied.

    Returns:
        pd.DataFrame: A copy of `df_target` with specified substring replacements applied.

    Raises:
        ValueError: If inputs are not DataFrames or required columns are missing.
    """
    if not isinstance(df_mapping, pd.DataFrame):
        raise ValueError("`df_mapping` must be a pandas DataFrame.")
    if not isinstance(df_target, pd.DataFrame):
        raise ValueError("`df_target` must be a pandas DataFrame.")
    if df_mapping.empty or df_target.empty:
        raise ValueError("empty df.")

    required_columns = {'col', 'val_old', 'val_new'}
    if not required_columns.issubset(df_mapping.columns):
        missing = required_columns - set(df_mapping.columns)
        raise ValueError(f"`df_mapping` is missing required columns: {missing}")

    df_result = df_target.copy()

    # * Clean and normalize the mapping data
    df_mapping = df_mapping.copy()
    df_mapping['col'] = df_mapping['col'].astype(str).str.strip()
    df_mapping['val_old'] = df_mapping['val_old'].astype(str).str.strip()
    df_mapping['val_new'] = df_mapping['val_new'].astype(str).str.strip()

    for col in df_mapping['col'].unique():
        if col in df_result.columns:
            replacements = df_mapping[df_mapping['col'] == col][['val_old', 'val_new']]
            df_result[col] = df_result[col].astype(str)

            for val_old, val_new in zip(replacements['val_old'], replacements['val_new']):
                pattern = re.compile(re.escape(val_old), flags=re.IGNORECASE)
                df_result[col] = df_result[col].str.replace(pattern, val_new, regex=True)

    return df_result

In [22]:
# # * import column mapping
df_codebook_values = (pd
        .read_excel(dir_data_codebooks/"codebook_values.xlsx")
        # * remove leading and trailing spaces
        .applymap(lambda x: x.strip() if isinstance(x, str) else x)
        # * ignore empty rows in code column
        .dropna(subset=["code"])
        # * rename columns for use in function
        .rename(columns={"de": "val_old", "en": "val_new"})
)

# * list all column codes should be searched in dataset
# * include duplicates, this list is added as a column later
stubs = df_codebook_values.code.to_list()

# * resulting full column names
found = hlp.find_cols(df_csv_condensed, stubs)

# * add these full names
df_codebook_values["col"]=found

# ! map values in dataset
_df = df_csv_condensed.copy()
df_csv_condensed = replace_substring_vectorized(df_mapping=df_codebook_values, df_target=_df)

# display(df_codebook_values[df_codebook_values.code == '[02.04]'])

### reduce columns

In [23]:
# # ! when key error: these columns have not beeen properly set, ALL must match 
df_csv_condensed = df_csv_condensed.loc[:,df_codebook_columns["col_final"].to_list()]

In [24]:
display(df_csv_condensed)
df_csv_condensed.columns.sort_values()

,[01.01] CTCAE,[01.02] Date,[01.03] Exercise-related,[02.02] Type,[02.03] Trigger,[02.04] Affected body parts,[03.01] Pain,[03.02] With hospitalization,[03.03] Medical follow-up treatment,[03.04] With delayed therapy protocol,...,[03.16] Death,[05.01] Therapy phase,[05.02] Group size,[05.03] Age,[05.04] Online,[05.05] As part of testing,[05.06] Setting,[05.07] Main motor skill,[05.08] Time point,[05.09] Training condition
1,2,Already present,No,Nausea/Vomiting,Physical strain|Medical therapy,Internal medicine type,Nein,No,No,No,...,No,Acute therapy,Individual,10 to 14 years,No,No,Hospital corridor,Coordination,2. Time point,Average
2,1,First occurrence,Yes,Pain,Coordination problems,Back|Buttocks,Ja,No,No,No,...,No,Acute therapy,Individual,6 to 9 years,No,No,Hospital corridor,Coordination,1. Time point,Good
3,2,Already present,No,Circulatory problems,Medical therapy,Internal medicine type,Nein,No,Yes,No,...,No,Acute therapy,Individual,10 to 14 years,No,No,Patients room,Coordination,2. Time point,Moderate
4,1,First occurrence,No,Nausea/Vomiting,Physical strain|Medical therapy,Internal medicine type,Nein,No,No,No,...,No,Acute therapy,Individual,<5 years,No,No,Patients room,Coordination,1. Time point,Average
5,3,First occurrence,Yes,Bone injuries,Fall event,Lower extremities,Ja,U,Yes,No,...,No,Aftercare,Individual,15 to 18 years,No,No,Gym,Coordination|Strength,1. Time point,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193,1,First occurrence,Yes,Circulatory problems,Physical strain,Full body,Nein,No,No,No,...,No,Acute therapy,Individual,6 to 9 years,No,No,Gym,Full body,2. Time point,Moderate
194,1,Already present,Yes,Circulatory problems,Physical strain,Full body,Nein,No,No,No,...,No,Acute therapy,Individual,6 to 9 years,No,No,nan,Coordination,2. Time point,Moderate
195,2,First occurrence,Yes,Nausea/Vomiting,Psychological strain|Medical therapy,Full body,Nein,No,No,No,...,Yes,Acute therapy,Individual,6 to 9 years,No,No,Gym,Flexibility,1. Time point,Moderate
196,1,First occurrence,Yes,Pain|Spontaneous painful bowel movement,Physical strain|Medical therapy,Intestine,Ja,No,No,No,...,No,Acute therapy,Individual,6 to 9 years,No,No,Gym,Full body,2. Time point,Moderate


Index(['[01.01] CTCAE', '[01.02] Date', '[01.03] Exercise-related',
       '[02.02] Type', '[02.03] Trigger', '[02.04] Affected body parts',
       '[03.01] Pain', '[03.02]  With hospitalization',
       '[03.03] Medical follow-up treatment',
       '[03.04] With delayed therapy protocol',
       '[03.05] Life-saving intervention', '[03.06] Increased care needs',
       '[03.07] With medication administration', '[03.08.01] Affected person',
       '[03.08] Occurrence of fear and uncertainty',
       '[03.09] Structural adjustment', '[03.10.01] Approver',
       '[03.10.02] OK to proceed',
       '[03.10] Assessment of the situation by expertise',
       '[03.11] Application RICE rule (Rest, Ice, Compression, Elevation)',
       '[03.12] With observation', '[03.13.01] Stop or Break', '[03.13] Stop',
       '[03.14] Adaptations', '[03.16] Death', '[05.01] Therapy phase',
       '[05.02] Group size', '[05.03] Age', '[05.04] Online',
       '[05.05] As part of testing', '[05.06] Setting',


## <a id='toc1_5_'></a>[💾 data for pivot use](#toc0_)

In [25]:
if ENABLE_UPDATE:
    # # * only show cols w/ >1 value
    df_csv_short = df_csv.loc[:, df_csv.apply(pd.Series.nunique) > 1]

    df_csv.to_csv(Path(dir_data_out/"df_csv.csv"),sep=";", index=False, encoding="utf-8-sig")
    df_csv_short.to_csv(Path(dir_data_out/"df_csv_short.csv"),sep=";", index=False, encoding="utf-8-sig")
    df_csv_condensed.sort_index(axis=1).to_csv(Path(dir_data_out/"df_csv_condensed.csv"),sep=";", index=False, encoding="utf-8-sig")

In [26]:
# for col in df_csv.columns.to_list()[8]:
#     x = col
#     x1=re.search(pattern=r"^\[\d{2}\.\d{2}(\.\d{2})?\]", string=x).group()
#     x3 = re.search(pattern=r"( \(choice=.*?\)+|\.\d$)", string=x).group()
#     x4 = re.search(pattern=r"\(choice=(.*?)\)", string=x).group(1)
#     x5 = re.search(pattern=r"\](.*?)\(choice=", string=x).group(1)
#     print(f"{x} | {x1} | {x4} | {x5}")